In [2]:
import numpy
import tqdm
import matplotlib
import gzip
import cv2
import IPython

In [3]:
# Steps for a CNN
# Input Layer, Repeat n times
#   Convolution | Applying 'smart' filters/kernels/weights on the data
#   Re-sizing or max pooling
# Multi-layer Perceptron
#   Fully-Connecter layer | Flatten / transform 3d to 2d (columns/rows into single column)
#   Output layer (Softmax Activation Function) | The probability 
# Calculating Loss

In [4]:
# Input Layer
# Show me the images!
'''
    Extract images by reading the file bytestream. 
    Reshape the read values into a 3D matrix of dimensions [m, h, w].
    Where m is the number of training examples.
'''
images_file_name='train-images-idx3-ubyte.gz'
number_of_images=1
image_height=28
image_width=28

print("Processing x images: ", number_of_images)

# Get the training data
with gzip.open(images_file_name) as bytestream:
    bytestream.read(16)
    buffer = bytestream.read( image_width * image_height * number_of_images )
    print("Buffer Size: ",len(buffer))
    
    d1_array_of_images = numpy.frombuffer(buffer, dtype=numpy.uint8).astype(numpy.float32)
    display("1D Array of Images, 1st entry",d1_array_of_images[0])
    
    d2_array_of_images = d1_array_of_images.reshape(number_of_images, image_height * image_width)
    display("2D Array of Images, 1st 175 elements of 1st row", d2_array_of_images[0,:175])
    display("Dimensions of 2D array:",d2_array_of_images.shape)
    
    d3_array_of_images = d1_array_of_images.reshape(number_of_images, image_height, image_width)
    display("3D Array of Images, 1st row, 1st set", d3_array_of_images[0][0])
    display("Dimensions of 3D array:",d3_array_of_images.shape)
    
    number_of_images, image_height, image_width = d3_array_of_images.shape
    
    _,an_image = cv2.imencode('.png', d3_array_of_images[0])
    IPython.display.display("1st Image! ", IPython.display.Image(data=an_image, width=224, height=224))
    

Processing x images:  1
Buffer Size:  784


'1D Array of Images, 1st entry'

0.0

'2D Array of Images, 1st 175 elements of 1st row'

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

...

'3D Array of Images, 1st row, 1st set'

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

'Dimensions of 3D array:'

(1, 28, 28)

'1st Image! '

In [5]:
# Get the labeled data
label_file_name='train-labels-idx1-ubyte.gz'
with gzip.open(label_file_name) as bytestream:
    bytestream.read(8)
    buffer = bytestream.read( number_of_images )
    print("Buffer Size: ",len(buffer))
    
    d1_array_of_labels = numpy.frombuffer(buffer, dtype=numpy.uint8).astype(numpy.int64)
    display("Label of the 1st image: ",d1_array_of_labels[0])

    d2_array_of_labels = d1_array_of_labels.reshape(number_of_images, 1)
    display("2D Array of labels, 1st row, 1st image", d2_array_of_labels[0])

Buffer Size:  1


'Label of the 1st image: '

5

'2D Array of labels, 1st row, 1st image'

array([5])

# Starting our first Hidden Layer
![hidden layer](images/Hidden_Layer.png)



In [7]:
##   Initializing 'smart' filters/kernels of the 1st (hidden) layer (beyond the image layer)

filter1_nuron_size = 8
filter1_depth = 1
filter1_columns = 5 
filter1_rows = 5 
filter1_dimensions= [filter1_nuron_size, filter1_depth, filter1_columns, filter1_rows]

# '''  Randomize the filters  
#     Initialize filter using a normal distribution with and a 
#     standard deviation inversely proportional the square root of the number of units
# '''
scale = 1.0
standard_deviation = scale/numpy.sqrt(numpy.prod(filter1_dimensions))
display("Standard Deviation of filter: ",standard_deviation)

filter1 = numpy.random.normal(loc=0, scale=standard_deviation, size=filter1_dimensions)
display("Base filter: ",filter1, "Base Filter Shape", numpy.shape(filter1))
display("Double checking base filter dimensions",filter1.shape)

filter1_bias = numpy.zeros((filter1.shape[0],1))
display("Let's skinny down and zero out filter1 (as a bias per filter): ", filter1_bias.shape, filter1_bias)


'Standard Deviation of filter: '

0.07071067811865475

'Base filter: '

array([[[[ 8.00389009e-02,  1.00975243e-01, -1.58887419e-03,
           1.73560884e-02,  1.09669763e-01],
         [-2.12085100e-02,  1.18187741e-02,  3.65133919e-02,
          -3.23199110e-02,  8.35411818e-03],
         [ 9.45987245e-02, -1.32308808e-01, -4.45794751e-02,
          -3.85440356e-02,  2.34687359e-02],
         [-2.86901972e-02,  8.31949048e-02, -9.91831110e-02,
          -9.48656620e-02, -2.29602932e-01],
         [-3.42802765e-02, -1.40294117e-01, -1.74743056e-01,
          -4.30015119e-02, -1.25518239e-02]]],


       [[[ 1.25996443e-01,  6.54658028e-03,  1.29096478e-01,
          -1.23183962e-01,  3.86475310e-02],
         [-6.76861798e-02,  6.85609252e-02, -2.03358012e-02,
           1.04416208e-01, -7.96828379e-03],
         [-1.13935905e-01, -8.96711270e-02, -2.04155076e-02,
          -3.15639420e-03,  8.53147521e-02],
         [-3.46561555e-02,  3.69570779e-02,  1.52056587e-02,
           3.15506474e-02,  4.92326644e-02],
         [-1.67421320e-03, -8.24349970e-02

'Base Filter Shape'

...

'Double checking base filter dimensions'

(8, 1, 5, 5)

"Let's skinny down and zero out filter1 (as a bias per filter): "

(8, 1)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [8]:
#Setting up our first convolution
filter_stride = 1 #i.e. number of elements to shift over when applying the filter

(num_of_images, pixels_per_image) = d2_array_of_images.shape
print("Shape of 2d image array: total images:",num_of_images, " pixels per image:", pixels_per_image)

filter_dimensions = int((image_height - filter1_columns ) / filter_stride)+1
display("Dimensions of output",filter_dimensions)

layer1_images_3d = numpy.zeros((filter1_nuron_size,filter_dimensions,filter_dimensions))

display("Output layer1 dimensions:", numpy.shape(layer1_images_3d))

Shape of 2d image array: total images: 1  pixels per image: 784


'Dimensions of output'

24

'Output layer1 dimensions:'

(8, 24, 24)

# Convolution of our filter over the input image!
![convolution gif](images/convolve-output.gif)

## The matrix math involved 
## Sum ( Filter * image-array ) + Filter bias


In [9]:
# Let's do it! (our first convolution that is)
# y1=0+5
# x1=25+5

for each_nureon_L1 in range(filter1_nuron_size): # Perform 8 times (i.e. 8 filters)
    current_column = output_column = 0
    while current_column + filter1_columns <= image_width: # Filter is 5x5, prevent going over edges.
        current_row = output_row = 0
        while current_row + filter1_rows <= image_height:
            layer1_images_3d[each_nureon_L1, output_column, output_row] = numpy.sum(
                filter1[each_nureon_L1] * d3_array_of_images[:,current_column: current_column + filter1_columns, 
                current_row: current_row + filter1_columns]
                ) + filter1_bias[each_nureon_L1]
            current_row += filter_stride
            output_row += 1
        current_column += filter_stride
        output_column += 1
#         display("Sum of entire output array:" ,numpy.sum(layer1_images_3d))
        
display("Output array shape after:" , numpy.shape(layer1_images_3d))
display("Did anything change in our array? ",layer1_images_3d[0])
_,an_image = cv2.imencode('.png', layer1_images_3d[0])
IPython.display.display("Output Image of N1", IPython.display.Image(data=an_image, width=224, height=224))
_,an_image = cv2.imencode('.png', layer1_images_3d[1])
IPython.display.display("Output Image of N2", IPython.display.Image(data=an_image, width=224, height=224))
_,an_image = cv2.imencode('.png', layer1_images_3d[2])
IPython.display.display("Output Image of N3", IPython.display.Image(data=an_image, width=224, height=224))
_,an_image = cv2.imencode('.png', layer1_images_3d[3])
IPython.display.display("Output Image of N4", IPython.display.Image(data=an_image, width=224, height=224))
_,an_image = cv2.imencode('.png', layer1_images_3d[4])
IPython.display.display("Output Image of N5", IPython.display.Image(data=an_image, width=224, height=224))
_,an_image = cv2.imencode('.png', layer1_images_3d[5])
IPython.display.display("Output Image of N6", IPython.display.Image(data=an_image, width=224, height=224))
_,an_image = cv2.imencode('.png', layer1_images_3d[6])
IPython.display.display("Output Image of N7", IPython.display.Image(data=an_image, width=224, height=224))
_,an_image = cv2.imencode('.png', layer1_images_3d[7])
IPython.display.display("Output Image of N8", IPython.display.Image(data=an_image, width=224, height=224))

'Output array shape after:'

(8, 24, 24)

'Did anything change in our array? '

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -3.76554717e-02,
        -3.54937366e-01, -1.52418921e+00, -4.56621740e+00,
        -8.12906697e+00, -1.34129526e+01, -3.32047390e+01,
        -4.99107714e+01, -5.71809917e+01, -4.40958736e+01,
        -5.27197288e+01, -8.09550450e+01, -9.00882526e+01,
        -6.55864855e+01, -2.62845812e+01, -4.35359512e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e

'Output Image of N1'

...

'Output Image of N4'

'Output Image of N5'

'Output Image of N6'

'Output Image of N7'

'Output Image of N8'

# The ReLU function
![ReLU Function](images/ReLU_Function.png)
Now, apply the ReLu function (Rectified Linear Unit) to the results.  
Values that are less than or equal to zero become zero and all positive values remain the same.


In [10]:
# Normalize the data
# display("Input for comparison ",layer1_images_3d[0])

layer1_images_3d[layer1_images_3d<=0] = 0

# display("Did anything change in our array? ",layer1_images_3d[0])
# display("Output array shape after x iterations:" , numpy.shape(layer1_images_3d))
# _,an_image = cv2.imencode('.png', layer1_images_3d[0])
# IPython.display.display("Output Image of N1", IPython.display.Image(data=an_image, width=224, height=224))
# _,an_image = cv2.imencode('.png', layer1_images_3d[1])
# IPython.display.display("Output Image of N2", IPython.display.Image(data=an_image, width=224, height=224))
# _,an_image = cv2.imencode('.png', layer1_images_3d[2])
# IPython.display.display("Output Image of N3", IPython.display.Image(data=an_image, width=224, height=224))
# _,an_image = cv2.imencode('.png', layer1_images_3d[3])
# IPython.display.display("Output Image of N4", IPython.display.Image(data=an_image, width=224, height=224))
# _,an_image = cv2.imencode('.png', layer1_images_3d[4])
# IPython.display.display("Output Image of N5", IPython.display.Image(data=an_image, width=224, height=224))
# _,an_image = cv2.imencode('.png', layer1_images_3d[5])
# IPython.display.display("Output Image of N6", IPython.display.Image(data=an_image, width=224, height=224))
# _,an_image = cv2.imencode('.png', layer1_images_3d[6])
# IPython.display.display("Output Image of N7", IPython.display.Image(data=an_image, width=224, height=224))
# _,an_image = cv2.imencode('.png', layer1_images_3d[7])
# IPython.display.display("Output Image of N8", IPython.display.Image(data=an_image, width=224, height=224))

# Max Pooling / Down sampling
Ideally you would apply additional filters/convolutions before this point.  
For brevity, we'll skip to down sampling

This will take the max of a 2x2 matrix (kernal_size x max_pool_stride)  and convert it to a single cell.

![max pooling](images/max_pooling_down_sampling.gif)

In [11]:
# Down sampling
#number_of_images, image_height, image_width
kernel_size = 2
max_pool_stride = 2
#Dimensions after down sampling / max pooling
max_pool_height = int((image_height - kernel_size)/max_pool_stride)+1
max_pool_width  = int((image_width  - kernel_size)/max_pool_stride)+1
print("Old height & width ", image_height, " & ",image_width)
print("New height & width ", max_pool_height, " & ", max_pool_width)

max_pool_images_3d = numpy.zeros((filter1_nuron_size, max_pool_height, max_pool_width))
  
for each_nureon_L1 in range(filter1_nuron_size): # Perform 8 times (i.e. 8 filters)
    current_rows = out_rows = 0
    # Slide the max pooling windows vertially across the image
    while current_rows + kernel_size <= max_pool_height:
        current_columns = out_columns = 0
        # Slide the max pooling windows horzontally across the image
        while current_columns + kernel_size <= max_pool_width:
            # Choose the max value within the window
#             print("On image: ",each_nureon_L1," height: ",current_rows, " width: ",current_columns)
            max_pool_images_3d[each_nureon_L1, out_rows, out_columns] = \
               numpy.max(layer1_images_3d[each_nureon_L1, current_rows:current_rows+kernel_size, current_columns:current_columns+kernel_size])
            current_columns += max_pool_stride
            out_columns += 1
        current_rows += max_pool_stride
        out_columns  += 1

display("Max Pooled / down sampled array (after 1st convolution)", numpy.shape(max_pool_images_3d))

Old height & width  28  &  28
New height & width  14  &  14


'Max Pooled / down sampled array (after 1st convolution)'

(8, 14, 14)

# Combine and Flatten all 8 instances of the previous layer into a 1d array.

![fully-connected layer](images/fully_connected_layer.png)

In [12]:
# Flatten the last layer before predictions are made
(max_pool_nodes, max_pool_width, max_pool_height) = max_pool_images_3d.shape
flattened_layer = max_pool_images_3d.reshape((1, max_pool_nodes * max_pool_width * max_pool_height))

(_, new_pixels_per_node) = numpy.shape(flattened_layer)
display("Prev Layer, flattened shape: ", numpy.shape(flattened_layer))
display("Prev Layer, flattened, first 10 pixals of layer",flattened_layer[0,0:10])

'Prev Layer, flattened shape: '

(1, 1568)

'Prev Layer, flattened, first 10 pixals of layer'

array([ 0.        ,  0.        ,  0.        ,  1.20636739, 33.80974233,
       29.24607199, 18.94879606,  0.        ,  0.        ,  0.        ])

# Predictions

A prediction is done with weights and biases

To calculate the prediction

preform a dot product of (previous flattened layer & the weights) + the biases

The result isn't in percentage form, and will need to go through further processing to make sense of them.

In [13]:
# Prediction (i.e. Does this nuron fire or not. One nuron per expected result)
#Now, fully connect (i.e. transpose into final layer of 10 predictions. One for each number we are trying to predict)

# # Initializing 'smart' weights
# '''
#     Initialize weights with a random normal distribution
# '''
# weight_multiplier = 0.01 #i.e. sigma
# weight1 = numpy.random.standard_normal(size=filter1_dimensions) * weight_multiplier
# display("The weights: ", weight1)

mu = 0
sigma = 0.01
input_size = new_pixels_per_node #YES! make this equal to number of bytes per node from previous flattening!
output_size = 10 # ==> number of predictions

# Weights size = pixels x filters (i.e. 196 x 10) Every pixel is going to have a % chance to be a 1-10
weights = numpy.random.normal(mu, sigma, (input_size, output_size))
display("What is the shape of the weights array: ", numpy.shape(weights))
display("What do the weights look like (1st entry): ",weights[0])

#Bias size 1 x 10 (Are these the biases per 1-10 prediction?)
biases = numpy.random.normal(mu, sigma, (1, output_size))
display("What is the shape of the MR bias array: ", numpy.shape(biases))
display("What does the bias look like (1st entry): ",biases[0])

# A rough estimate of predictions that still need to be run through a 'soft max' function to make more sense
connected_layer_predictions = numpy.dot(flattened_layer, weights) + biases
display("What is the shape of the connected_layer_predictions array: ", numpy.shape(connected_layer_predictions))
display("What does the connected_layer_predictions look like: ",connected_layer_predictions)

'What is the shape of the weights array: '

(1568, 10)

'What do the weights look like (1st entry): '

array([-3.34158220e-05, -1.61998907e-02, -1.28964930e-02,  1.34449107e-02,
        1.13523178e-02,  7.93244973e-03,  8.71804621e-04, -8.28962569e-03,
        1.86347100e-03,  5.74132610e-03])

'What is the shape of the MR bias array: '

...

array([-0.00839651,  0.00537921, -0.00015442, -0.0133933 , -0.01703193,
       -0.01131082, -0.01152201, -0.02705437, -0.02258369, -0.00739881])

'What is the shape of the connected_layer_predictions array: '

(1, 10)

'What does the connected_layer_predictions look like: '

array([[-1.7999346 ,  0.94741722,  1.96200755, -0.85250558, -0.20808445,
        -2.21651575,  1.02659415, -0.80018013,  2.68303293,  0.45662072]])

# Create predictions
Leveraging the "Softmax Activation Function"
![Softmax Activation Function](images/softmax_activation_function.png)

In [14]:

# Do a soft max (get the numbers into percentage form)
exp_of_conneted_layer_predictions = numpy.exp(connected_layer_predictions)

predictions = exp_of_conneted_layer_predictions / numpy.sum(exp_of_conneted_layer_predictions)
display("Final Preditions! ", predictions)

# exponent_of_weight1 = numpy.exp(weight1)
# predictions = exponent_of_weight1/numpy.sum(exponent_of_weight1)

# display("Final Predictions: ", predictions)

'Final Preditions! '

array([[0.0053927 , 0.08413297, 0.23205835, 0.01390816, 0.02649343,
        0.00355539, 0.09106517, 0.01465529, 0.47723748, 0.05150104]])

In [15]:

# Convert label data into binary on/off switch of which digit is present
# i.e. for the number 5, flip the entry for the 5th digit in the array of 10 numbers [0,0,0,0,0,1,0,0,0,0]
# for i in range(batch_size):

total_images_to_predict = 10
processing_this_image_in_array = 0
array_of_which_image_present = numpy.eye(total_images_to_predict)[int(d2_array_of_labels[processing_this_image_in_array])].reshape(total_images_to_predict, 1) # convert label to one-hot
display("Which Image Present: ",numpy.shape(array_of_which_image_present), array_of_which_image_present)

# # My Calculated loss
# # [0,0,0,0,0,1,0,0,0,0]
# inverse_array_of_label = [1],[1],[1],[1],[1],[0],[1],[1],[1],[1]
# my_loss_percentage = 10*numpy.sum(inverse_array_of_label * predictions)
# display("My loss %: ", my_loss_percentage)

# array_of_label = [0],[0],[0],[0],[0],[1],[0],[0],[0],[0]
# display("My array",array_of_label)
# display("Shape of my array ", numpy.shape(array_of_label))

# Actual log analysis of the loss
display("log: ", numpy.log(predictions))
loss_logarithm = -numpy.sum(array_of_which_image_present * numpy.log(predictions))
display("The loss logarithm: ", loss_logarithm)


'Which Image Present: '

(10, 1)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.]])

'log: '

array([[-5.22270857, -2.47535674, -1.46076642, -4.27527955, -3.63085842,
        -5.63928972, -2.39617982, -4.22295409, -0.73974104, -2.96615325]])

'The loss logarithm: '

33.0292876202119

# Backwards Propagation!

In [16]:
# Backwards propagation
## Reverse Fully Conneted (1x10 matrix of probabilities. % prediction per label)
#  <== 'predictions' matrix
## Reverse Flatten


# Predictions applied to weights and biases
learning_rate = 0.01

# Transpose flattened layer and multiple by predictions.
loss_of_last_layer = flattened_layer.T @ predictions # ==> 1586 x 10
display("loss_of_last_layer shape",numpy.shape(loss_of_last_layer))

loss_of_bias = biases * predictions
display("loss_of_bias shape",numpy.shape(loss_of_bias))

loss_input = (weights @ predictions.T).reshape(flattened_layer.shape)
display("loss_input shape",numpy.shape(loss_input))

weights -= learning_rate * loss_of_last_layer
biases  -= learning_rate * loss_of_bias


'loss_of_last_layer shape'

(1568, 10)

'loss_of_bias shape'

(1, 10)

'loss_input shape'

(1, 1568)

In [17]:
# Reverse Flatten (convert flattened losses back into ouptput of filters i.e. previous layer)
loss_input_unflat = loss_input.reshape(max_pool_images_3d.shape)
display("loss_input_unflat shape",numpy.shape(loss_input_unflat))


'loss_input_unflat shape'

(8, 14, 14)

In [18]:
# Reverse Max Pooling

def find_max(pooling_size, feature_map_index, row_start, col_start):
        """
        :param row_start: int
        :param col_start: int
        :return: max value within pool range
        """
        activated_matrix = layer1_images_3d
        activated_matrix_shape = activated_matrix.shape
        activated_matrix_height = activated_matrix_shape[1]
        activated_matrix_width = activated_matrix_shape[2]

        maximum = 0
        maximum_location = (row_start, col_start)
        for y in range(0, pooling_size):
            for x in range(0, pooling_size):
                if (
                    row_start + y < activated_matrix_height
                    and col_start + x < activated_matrix_width
                ):
                    if (
                        activated_matrix[feature_map_index][y + row_start][
                            x + col_start
                        ]
                        > maximum
                    ):
                        maximum = activated_matrix[feature_map_index][y + row_start][
                            x + col_start
                        ]
                        maximum_location = (y + row_start, x + col_start)

        return maximum, maximum_location

kernel_size = 2
max_pool_stride = 2
#Dimensions after down sampling / max pooling

_, rev_max_pool_height, rev_max_pool_width = layer1_images_3d.shape

print("Old shape ", max_pool_images_3d.shape )
print("New shape ", layer1_images_3d.shape )

rev_max_pool_images_3d = numpy.zeros((filter1_nuron_size, rev_max_pool_height, rev_max_pool_width))
  
for feature_map_index in range(filter1_nuron_size): # Perform 8 times (i.e. 8 filters)
    current_rows = out_rows = 0
    # Slide the max pooling windows vertially across the image
    while current_rows + kernel_size <= rev_max_pool_height:
        current_columns = out_width = 0
        # Slide the max pooling windows horzontally across the image
        while current_columns + kernel_size <= rev_max_pool_width:
            gradient_to_extract = rev_max_pool_images_3d[feature_map_index][current_rows][current_columns]
            _ , max_location = find_max(max_pool_stride, feature_map_index, current_rows, current_rows)
            
            max_row, max_col = max_location
            
            rev_max_pool_images_3d[feature_map_index][max_row][max_col] = gradient_to_extract
            current_columns += 1
        current_rows += 1
        current_columns = 0

display("Results (shape): ", rev_max_pool_images_3d.shape) # d_loss_d_input

Old shape  (8, 14, 14)
New shape  (8, 24, 24)


'Results (shape): '

(8, 24, 24)

In [19]:
# Reverse Convolution
output_matrix_depth, output_matrix_height, output_matrix_width = rev_max_pool_images_3d.shape

rev_filter1 = numpy.zeros(filter1.shape)
#filter1_bias

def apply_d_output(input_matrix, gradient_at_location, current_output_row, current_output_col):
    filter_partials_to_return = numpy.zeros(filter1[0].shape)

    for depth in range(filter1_depth):
        for row in range(filter1_rows):
            for col in range(filter1_columns):
                input_value = input_matrix[depth][current_output_row + row][
                    current_output_col + col
                ]
                filter_partials_to_return[depth][row][col] = (
                    gradient_at_location * input_value
                )

    return filter_partials_to_return

# walk each kernel over original input
for current_feature_map in range(output_matrix_depth):
    for current_row in range(output_matrix_height):
        for current_col in range(output_matrix_width):
            current_gradient = rev_max_pool_images_3d[current_feature_map][current_row][current_col]
            applied_gradient = apply_d_output(d3_array_of_images, current_gradient, current_row, current_col)
            rev_filter1[current_feature_map] += applied_gradient

display("Output filter shape:", rev_filter1.shape)
# update kernels/filter by learning rate and d_loss_d_filters
filter1 -= learning_rate * rev_filter1
        

'Output filter shape:'

(8, 1, 5, 5)